In [1]:
import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from dataloader import CustomDataset
from submission import get_model, eval_transform, team_id, team_name, email_address



In [2]:
evalset = CustomDataset(root='D:/NYU\DeepLearning\hw\Final\dataset', split="val", transform=eval_transform)
evalloader = torch.utils.data.DataLoader(evalset, batch_size=256, shuffle=False, num_workers=2)

net = get_model()
checkPointDir = 'D:/NYU/DeepLearning/hw/Final_git/dl_project/test_DL/model.pth'
checkpoint = torch.load(checkPointDir)
net.load_state_dict(checkpoint)
net = net.cuda()

net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in evalloader:
        images, labels = data

        images = images.cuda()
        labels = labels.cuda()

        outputs = net(images)
        #print(torch.max(outputs.data, 1))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(f"Team {team_id}: {team_name} Accuracy: {(100 * correct / total):.2f}%")
print(f"Team {team_id}: {team_name} Email: {email_address}")

Team 10: dl10 Accuracy: 14.58%
Team 10: dl10 Email: wl2337@nyu.edu


In [12]:
# Feel free to modifiy this file. 
# It will only be used to verify the settings are correct 

import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from dataloader import CustomDataset
from submission import *

# parser = argparse.ArgumentParser()
# parser.add_argument('--checkpoint-dir', type=str)
# args = parser.parse_args()

train_transform = transforms.Compose([
    transforms.ToTensor(),
])


from submission import get_model, eval_transform, team_id, team_name, email_address

unLabeledSet = CustomDataset(root='D:/NYU\DeepLearning\hw\Final\dataset', split="unlabeled", transform=train_transform)
unLabeledLoader = torch.utils.data.DataLoader(unLabeledSet, batch_size = 8, shuffle=False, num_workers = 2)




with torch.no_grad():
    net.eval()
    newlyLabeledList = []
    requestList = []
    for iC, data in enumerate(unLabeledLoader):

        # for data in evalloader:
        images, labels = data

        images = images.cuda()
        labels = labels.cuda()

        outputs = net(images)
        #Smallest Margin Uncertainty
        tk_v,tk_i = torch.topk(outputs,2)

        
        
        #margin = torch.max(outputs, 1)[0] - torch.min(outputs, 1)[0]
        margin = tk_v[:,0] - tk_v[:,1]

        maxMarginPosition = torch.argmax(margin)
        pickedUnlabelled_img = images[maxMarginPosition]
        pickedUnlabelled_lbl = tk_i[maxMarginPosition][0]

        names_start = iC*8
        #print(maxMarginPosition)
        if(maxMarginPosition==0):
            request = names_start+1
        else:
            request = names_start
        #print(pickedUnlabelled_img)
        tempTouple = (names_start+maxMarginPosition, pickedUnlabelled_lbl,)
        #print(tempTouple)
        newlyLabeledList.append(tempTouple)
        #print(pickedUnlabelled_img.tolist())
        requestList.append(request)





In [13]:
len(newlyLabeledList)

64000

In [14]:
len(requestList)

64000

In [20]:
with open('request_10.csv', 'w') as f:
    for idx in requestList:
        if(idx/8 >= 12800): break
        print(f"{idx}.png,",file = f)
 

In [28]:
with open('newlyLabeled.txt', 'w') as f:
    for tempTouple in newlyLabeledList:
        name,label = tempTouple
        print(f"{name.item()}.png,{label.item()}",file = f)


In [33]:
labelDir = r"D:\NYU\DeepLearning\hw\Final\dataset\train_label_tensor.pt"

In [34]:
givenlabels = torch.load(labelDir)

In [35]:
givenlabels

tensor([289, 289, 289,  ...,  93,  93,  93])